In [1]:
from  syft.core.hooks  import TorchHook
from syft.core.workers import VirtualWorker, SocketWorker
import torch
import torch.nn as nn
from torch.autograd import Variable as Var
import torch.optim as optim

# this is our hook
hook = TorchHook(local_worker=SocketWorker(id=0, port=8186))

Starting Socket Worker...
Ready!
Hooking into Torch...
Overloading complete.


In [2]:
remote_worker_metadata={"id": 1,
                        "type":"syft.core.workers.socket.SocketWorker",
                        "hostname":"localhost",
                        "port":8205}

def init_worker_pointer(hook, metadata):
    
    if(metadata['type'] == "syft.core.workers.socket.SocketWorker"):
        remote_client = SocketWorker(hook=hook,
                                     id=metadata['id'],
#                                      hostname=metadata['hostname'],
                                     port=metadata['port'],
                                     is_pointer=True)
        return remote_client
hook.local_worker.add_worker(init_worker_pointer(hook, remote_worker_metadata))

Attaching Pointer to Socket Worker...


In [3]:
hook.local_worker.listen(1)

Received Command From: ('127.0.0.1', 58764)


In [4]:
hook.local_worker._objects[8494475226].get_()

Variable containing:
 0  0
 0  1
[torch.FloatTensor of size 2x2]

In [2]:
def create_tensor_pointer(tensor_id, tensor_type, remote_worker_id, remote_worker_init_metadata={}):
    
    if remote_worker_id not in hook.local_worker._known_workers:
        print("adding worker")
        remote_client = init_worker_pointer(hook,remote_worker_init_metadata)
        hook.local_worker.add_worker(remote_client)

    ptr = hook._assemble_result_pointer({"id":tensor_id, "owners":[remote_worker_id]}, tensor_type, None, None)
    ptr.is_pointer = True
    return ptr

tensor_id = 6555669521
tensor_type = "torch.FloatTensor"


x = create_tensor_pointer(tensor_id, "torch.FloatTensor", remote_worker_metadata['id'], remote_worker_metadata)

adding worker
Attaching Pointer to Socket Worker...


adding worker
Attaching Pointer to Socket Worker...


adding worker


NameError: name 'init_worker_pointer' is not defined

In [5]:
y = x + x

In [6]:
y.get()


 0  0
 0  2
[torch.FloatTensor of size 2x2]

In [7]:
x.get()


 0  0
 0  1
[torch.FloatTensor of size 2x2]

In [ ]:
# FEDERATED LEARNING SERVER

# this server listens for tensors to be submitted by clients
# for training of a model

import asyncio
import websockets
import json

async def hello(websocket, path):
    json_message = await websocket.recv()
    message_obj = json.loads(json_message)
    input_id = message_obj['dataset']['input']
    target_id = message_obj['dataset']['target']
    print(f"<  {input_id} {target_id}")

    greeting = f"Hello {message_obj}!"

    await websocket.send(greeting)
    print(f"> {greeting}")

start_server = websockets.serve(hello, 'localhost', 8766)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

<  1396140055 949881783
> Hello {'dataset': {'input': 1396140055, 'target': 949881783}, 'worker': '{"hostname": "localhost", "port": 8184}'}!
<  5646722488 6328116976
> Hello {'dataset': {'input': 5646722488, 'target': 6328116976}, 'worker': '{"hostname": "localhost", "port": 8184, "id": 0}'}!
